# Enjoyable, Reproducible Software Environments

This notebook introduces **[Pixi](https://pixi.sh)**, a *cross-platform package management tool to manage software environments in a reproducible way*.

We will learn how to:

- Have an **enjoyable package-management experience 😄**
- **Create cross-platform reproducible software environments** for microCT analysis and visualization 🔬
- **Forgo dependency hell 👿**

There are three sections addressing,

1. What is Pixi
    - A **fast** package manager written in **Rust** and distributed as a single binary
    - Utilizes packages from both **conda-forge** and **PyPI** by default
    - Supports **Python, R, C/C++, Rust, Ruby** and many other languages
2. What makes Pixi different
    - Each project's dependencies are **local and self-contained**
    - **Resolves relationship between conda and PyPI packages**
    - **Automatic lockfiles** produce reproducible environments across operating systems (without Docker!).
4. Basic usage
    - **Start a project**
    - **Add a dependency**
    - **Define a task**

## Get pixi and this repository

### 1. Get the source code

Either download and unpack the [repository source code ZIP file](https://github.com/thewtex/als-user-meeting-2024/archive/refs/heads/main.zip) or clone the repository with Git:

```bash
git clone https://github.com/thewtex/als-user-meeting-2024.git
cd als-user-meeting-2024
```

### 2. Install pixi

On macOS, Linux:

```bash
curl -fsSL https://pixi.sh/install.sh | bash
```

For Windows Powershell:

```powershell
iwr -useb https://pixi.sh/install.ps1 | iex
```

And restart your shell.

### 3. Start the tutorial

Change to the directory with the tutorial sources and run:

```bash
pixi run start
```

This will install all dependencies and start Jupyter.

## What is Pixi?

A few excerpts from [this blog post on Pixi for Scientists](https://prefix.dev/blog/pixi_for_scientists).

![Pixi history](./figures/pixi_history.svg)

> Pixi is a new package manager built on the foundation of the conda and conda-forge ecosystem. Created by the team behind mamba, Pixi leverages the extensive conda-forge distribution, which includes a vast array of scientific software packages such as Python, R, C/C++ libraries, NumPy, SciPy, and many others.

At its core, Pixi aims to solve three critical challenges in scientific software development:

1. **Collaboration**: Enabling seamless sharing and reproduction of research code
2. **Reproducibility**: Ensuring consistent execution across different machines
3. **Performance**: Maximizing the utilization of available hardware resources

#### Pixi: High Reproducibility with Low Effort

![Pixi graph](./figures/pixi-graph.svg)

In [1]:
!pixi --help

Pixi [version 0.27.1] - Developer Workflow and Environment Management for
Multi-Platform, Language-Agnostic Projects.

Pixi is a versatile developer workflow tool designed to streamline the
management of your project's dependencies, tasks, and environments.
Built on top of the Conda ecosystem, Pixi offers seamless integration with the
PyPI ecosystem.

Basic Usage:
    Initialize pixi for a project:
    $ pixi init
    $ pixi add python numpy pytest

    Run a task:
    $ pixi add task test 'pytest -s'
    $ pixi run test

Found a Bug or Have a Feature Request?
Open an issue at: https://github.com/prefix-dev/pixi/issues

Need Help?
Ask a question on the Prefix Discord server: https://discord.gg/kKV8ZxyzY4

For more information, see the documentation at: https://pixi.sh

Usage: pixi [OPTIONS] <COMMAND>

Commands:
  init         Creates a new project
  add          Adds dependencies to the project [aliases: a]
  remove       Removes dependencies from the project [aliases: rm]
  install   

Information about:

- **The current platform**: your system and features of your system
- **The current project**: where the configuration of the project and all packages live
- **The current software environment**: what packages and tasks are involved in your project

can be found with `pixi info`.

In [2]:
!pixi info

      Pixi version: 0.27.1
          Platform: linux-aarch64
  Virtual packages: __unix=0=0
                  : __linux=5.15.153.1=0
                  : __glibc=2.39=0
                  : __archspec=1=aarch64
         Cache dir: /home/matt/.cache/rattler/cache
      Auth storage: /home/matt/.rattler/credentials.json

Project
------------
              Name: als-user-meeting-2024
           Version: 1.0.0
     Manifest file: /home/matt/src/als-user-meeting-2024/pixi.toml
  Config locations: 
      Last updated: 12-08-2024 16:35:04

Environments
------------
       Environment: default
          Features: default
          Channels: conda-forge
  Dependency count: 2
      Dependencies: jupyterlab, python
  Target platforms: osx-64, linux-64, win-64, osx-arm64, linux-aarch64
             Tasks: start




All the *package dependencies live locally* in your project folder.

Your local folder is *a self-contained software environment* in the **.pixi** directory.

In [3]:
%ldir .pixi

drwxr-xr-x 3 matt 4096 Aug 12 17:30 envs/


Inside the *pixi.lock* files, pixi defines

1. **All the transitive dependences** required for a project's packages.
2. Dependency resolution that **includes relationships between conda and PyPI packages**.
3. Dependency resolution **per-platform**.

In [4]:
%less pixi.lock

version: 5
environments:
  default:
    channels:
    - url: https://conda.anaconda.org/conda-forge/
    packages:
      linux-64:
      - conda: https://conda.anaconda.org/conda-forge/linux-64/_libgcc_mutex-0.1-conda_forge.tar.bz2
      - conda: https://conda.anaconda.org/conda-forge/linux-64/_openmp_mutex-4.5-2_gnu.tar.bz2
      - conda: https://conda.anaconda.org/conda-forge/noarch/anyio-4.4.0-pyhd8ed1ab_0.conda
      - conda: https://conda.anaconda.org/conda-forge/noarch/argon2-cffi-23.1.0-pyhd8ed1ab_0.conda
      - conda: https://conda.anaconda.org/conda-forge/linux-64/argon2-cffi-bindings-21.2.0-py312h98912ed_4.conda
      - conda: https://conda.anaconda.org/conda-forge/noarch/arrow-1.3.0-pyhd8ed1ab_0.conda
      - conda: https://conda.anaconda.org/conda-forge/noarch/asttokens-2.4.1-pyhd8ed1ab_0.conda
      - conda: https://conda.anaconda.org/conda-forge/noarch/async-lru-2.0.4-pyhd8ed1ab_0.conda
      - conda: https://conda.anaconda.org/conda-forge/noarch/attrs-24.2.0-pyh71513ae_

We can observe the direct and transitive dependencies of our project's packages with `pixi tree`.

In [5]:
!pixi tree

├── jupyterlab 4.2.4 
│   ├── async-lru 2.0.4 
│   │   ├── python 3.12.5 
│   │   │   ├── bzip2 1.0.8 
│   │   │   │   └── libgcc-ng 14.1.0 
│   │   │   │       └── _openmp_mutex 4.5 
│   │   │   │           └── libgomp 14.1.0 
│   │   │   ├── ld_impl_linux-aarch64 2.40 
│   │   │   ├── libexpat 2.6.2 
│   │   │   │   └── libgcc-ng 14.1.0 (*)
│   │   │   ├── libffi 3.4.2 
│   │   │   │   └── libgcc-ng 14.1.0 (*)
│   │   │   ├── libgcc-ng 14.1.0 (*)
│   │   │   ├── libnsl 2.0.1 
│   │   │   │   └── libgcc-ng 14.1.0 (*)
│   │   │   ├── libsqlite 3.46.0 
│   │   │   │   ├── libgcc-ng 14.1.0 (*)
│   │   │   │   └── libzlib 1.3.1 
│   │   │   │       └── libgcc-ng 14.1.0 (*)
│   │   │   ├── libuuid 2.38.1 
│   │   │   │   └── libgcc-ng 14.1.0 (*)
│   │   │   ├── libxcrypt 4.4.36 
│   │   │   │   └── libgcc-ng 14.1.0 (*)
│   │   │   ├── libzlib 1.3.1 (*)
│   │   │   ├── ncurses 6.5 
│   │   │   │   └── libgcc-ng 14.1.0 (*)
│   │   │   ├── openssl 3.3.1 
│   │   │   │   ├── ca-certificates 20

If we remove the software environment in *.pixi*, it will be re-installed with `pixi install` directly or by running a task that requires the environment.

In [6]:
# Remove the project software environments
!pixi clean

  removed /home/matt/src/als-user-meeting-2024/.pixi/envs                       

In [7]:
# Use --no-progress to avoid pretty progress intended for the terminal
!pixi --no-progress install

✔ The default environment has been installed.


Note that restoring or recreating similar software environments is extremely fast because *pixi will re-use packages based on their name, version, and hash* from a global cache.

## Pixi basics

Let's run through the [pixi basics tutorial](https://pixi.sh/latest/basic_usage/).

Initialize a new project and navigate to the project directory.

*Open a new shell prompt.*

```shell
pixi init pixi-hello-world
cd pixi-hello-world
```

Add the dependencies you would like to use.

```shell
pixi add python
```

Create a file named *hello_world.py* in the directory and paste the following code into the file.

```python
def hello():
    print("Hello World, to the new revolution in package management.")

if __name__ == "__main__":
    hello()
```

Run the code inside the environment.

```shell
pixi run python hello_world.py
```

You can also put this run command in a **task**.

```shell
pixi task add hello python hello_world.py
```

After adding the task, you can run the task using its name.

```shell
pixi run hello
```

Use the shell command to activate the environment and start a new shell in there.

```shell
pixi shell
python
exit()
```

You've just learned the basic features of pixi:

1. initializing a project
2. adding a dependency.
3. adding a task, and executing it.
4. running a program.

More information can be found in the [pixi documentation](https://pixi.sh/latest/).

Happy hacking!